In [17]:
import findspark
import pandas as pd
from data_preprocessing import get_cleaned_data, get_cleaned_data_final
findspark.init()
from sklearn.datasets import load_iris

In [2]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
spark=SparkSession.builder\
    .master("local[*]")\
    .appName("Apriori")\
    .getOrCreate()

In [3]:
sc=spark.sparkContext

In [21]:
data = get_cleaned_data_final(convert_categorical=True)

In [22]:
data_rdd = spark.createDataFrame(data.head(100_000)).rdd

In [23]:
# (key : class label , value : {attribute1 : value1, attribute2 : value2})
def mapper(row):
	dict = row.asDict()
	dict['count'] = 1
	result = {}
	for k,v in dict.items():
		if k != 'TARGET':
			result[k] = v
	return (dict['TARGET'], result)

def reducer(old, new):
	for k,v in new.items():
		old[k] = old[k] + v
	return old

mean = data_rdd.map(mapper).reduceByKey(reducer).collect()


In [28]:
print(mean)
for r in mean:
	for k,v in r[1].items():
		r[1][k] = v / r[1]['count']
print(mean)

[(0, {'NAME_CONTRACT_TYPE': 8908, 'CODE_GENDER': 30560, 'FLAG_OWN_CAR': 31265, 'FLAG_OWN_REALTY': 63803, 'AMT_INCOME_TOTAL': 15094005738.300001, 'AMT_CREDIT': 54210647961.0, 'AMT_ANNUITY': 2455231216.0173645, 'NAME_TYPE_SUITE': 479191, 'NAME_INCOME_TYPE': 428339, 'NAME_EDUCATION_TYPE': 292009, 'NAME_FAMILY_STATUS': 134930, 'NAME_HOUSING_TYPE': 117648, 'REGION_POPULATION_RELATIVE': 1915.664856000001, 'DAYS_BIRTH': -1478846158, 'DAYS_EMPLOYED': 5951784104, 'DAYS_REGISTRATION': -458159148.0, 'DAYS_ID_PUBLISH': -276870697, 'FLAG_EMP_PHONE': 75040, 'FLAG_WORK_PHONE': 18117, 'FLAG_CONT_MOBILE': 91655, 'FLAG_PHONE': 25845, 'FLAG_EMAIL': 5148, 'CNT_FAM_MEMBERS': 197789.15266545044, 'REGION_RATING_CLIENT_W_CITY': 186261, 'WEEKDAY_APPR_PROCESS_START': 283139, 'HOUR_APPR_PROCESS_START': 1108727, 'LIVE_REGION_NOT_WORK_REGION': 3590, 'REG_CITY_NOT_LIVE_CITY': 7022, 'LIVE_CITY_NOT_WORK_CITY': 16210, 'ORGANIZATION_TYPE': 2708046, 'EXT_SOURCE_2': 47960.197966215914, 'OBS_60_CNT_SOCIAL_CIRCLE': 128690.